In [1]:
# Source: #https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import calendar
import en_core_web_sm
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel, TfidfModel
from gensim.utils import simple_preprocess
import glob
import json
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import numpy as np
import os
import pyLDAvis
import pyLDAvis.gensim
import pandas as pd
import regex
import seaborn as sns
import spacy
from collections import Counter
from datetime import date, datetime
import math
from nltk.corpus import stopwords
from tqdm import tqdm, tqdm_notebook
import warnings
from wordcloud import WordCloud

sns.set_theme(style="whitegrid")
nlp = en_core_web_sm.load()
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/fuzzysha/software/mi

In [2]:
# set display options for pandas dataframes

pd.options.display.max_columns=2000
pd.options.display.max_rows=2000
pd.options.display.max_colwidth=10000
pd.options.display.max_seq_items=2000

# read in three CSV files into pandas dataframes
df_news_true=pd.read_csv('data/DataSet_Misinfo_TRUE.csv',index_col=False)
df_news_fake=pd.read_csv('data/DataSet_Misinfo_FAKE.csv')
df_news_rpsub=pd.read_csv('data/EXTRA_RussianPropagandaSubset.csv')

In [3]:
df_news_fake['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [4]:
# Function to load data from a JSON file
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return data


# Function to write data to a JSON file
def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


# Function for lemmatizing text
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in tqdm_notebook(texts):
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out


# Lemmatize the text in a dataframe
lemmatized_texts = lemmatization(df_news_fake[df_news_fake['text'].isna()==False]['text'])
print(lemmatized_texts[0:2])

# Function for generating words from text
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final


# Generate words from the lemmatized text
data_words = gen_words(lemmatized_texts)

print(data_words[0:2])

/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


  0%|          | 0/43642 [00:00<?, ?it/s]

['just couldn t wish leave instead give shout enemy hater very dishonest fake news medium former reality show star just job couldn t rapidly grow strong smart want wish friend supporter enemy hater even very dishonest happy tweet great year rapidly grow strong smart want wish friend supporter enemy hater even very dishonest happy great year tweet go about as welll d expect kind president send greeting despicable petty infantile gibberish only lack decency win t even allow rise gutter long enough wish american citizen happy new year one like impeachment make great year also accept regain control 2017do hear talk When include many people hate wonder Why hate use word hater year wish @marlene399 just say happy new year pollitt s tweet include many enemy fight lose so badly just don know love new s do year direct message enemy hater anniversary holiday tweet clearly presidential How long work become s always only difference last few year filter break apart teenager use term hater s fucking

In [5]:
# Creating bigram and trigram phrases
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

# Creating bigram and trigram models
bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# Function to make bigrams
def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

# Function to make trigrams
def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

# Creating bigrams for the preprocessed text data
data_bigrams = make_bigrams(data_words)

# Creating trigrams for the preprocessed text data
data_bigrams_trigrams = make_trigrams(data_bigrams)

print(data_bigrams_trigrams[:2])


[['just', 'couldn', 'wish', 'leave', 'instead', 'give', 'shout', 'enemy', 'hater', 'very', 'dishonest', 'fake', 'news', 'medium', 'former', 'reality', 'show', 'star', 'just', 'job', 'couldn', 'rapidly', 'grow', 'strong', 'smart', 'want', 'wish', 'friend', 'supporter', 'enemy', 'hater', 'even', 'very', 'dishonest', 'happy', 'tweet', 'great', 'year', 'rapidly', 'grow', 'strong', 'smart', 'want', 'wish', 'friend', 'supporter', 'enemy', 'hater', 'even', 'very', 'dishonest', 'happy', 'great', 'year', 'tweet', 'go', 'about', 'as', 'welll', 'expect', 'kind', 'president', 'send', 'greeting', 'despicable', 'petty', 'infantile', 'gibberish', 'only', 'lack', 'decency', 'win', 'even', 'allow', 'rise', 'gutter', 'long', 'enough', 'wish', 'american', 'citizen', 'happy', 'new', 'year', 'one', 'like', 'impeachment', 'make', 'great', 'year', 'also', 'accept', 'regain', 'control', 'do', 'hear', 'talk', 'when', 'include', 'many', 'people', 'hate', 'wonder', 'why', 'hate', 'use', 'word', 'hater', 'year', 

In [6]:
# Create the id2word dictionary
id2word = corpora.Dictionary(data_bigrams_trigrams)

# Create corpus
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]

# Apply tf-idf
tfidf = TfidfModel(corpus, id2word=id2word)

# Remove low tf-idf value words and those missing from tf-idf model
low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in tqdm(range(len(corpus))):
    bow = corpus[i]
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

# Train LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

# Get the topic distribution for a test document
test_doc = corpus[-1]
vector = lda_model[test_doc]
print(vector)

# Sort the topic distribution in descending order
def sort_topic_distribution(sub_li):
    sub_li.sort(key=lambda x: x[1])
    sub_li.reverse()
    return sub_li

new_vector = sort_topic_distribution(vector)
print(new_vector)

# Save and load the LDA model
lda_model.save("test_model_fake.model")
new_model = gensim.models.ldamodel.LdaModel.load("test_model_fake.model")

# Get the topic distribution for the same test document using the loaded model
test_doc = corpus[-1]
vector = new_model[test_doc]
print(vector)

# Sort the topic distribution in descending order
new_vector = sort_topic_distribution(vector)
print(new_vector)

100%|██████████| 43642/43642 [00:52<00:00, 824.31it/s] 


[(0, 0.23572388), (1, 0.066168495), (2, 0.0893781), (3, 0.021639204), (4, 0.25399992), (5, 0.18243818), (6, 0.022080697), (7, 0.049392525), (8, 0.03740779), (9, 0.041771173)]
[(4, 0.25399992), (0, 0.23572388), (5, 0.18243818), (2, 0.0893781), (1, 0.066168495), (7, 0.049392525), (9, 0.041771173), (8, 0.03740779), (6, 0.022080697), (3, 0.021639204)]
[(0, 0.23572032), (1, 0.06616854), (2, 0.08937754), (3, 0.021639211), (4, 0.2540009), (5, 0.18243863), (6, 0.022081727), (7, 0.04939416), (8, 0.037407793), (9, 0.041771173)]
[(4, 0.2540009), (0, 0.23572032), (5, 0.18243863), (2, 0.08937754), (1, 0.06616854), (7, 0.04939416), (9, 0.041771173), (8, 0.037407793), (6, 0.022081727), (3, 0.021639211)]


In [7]:
#enables the display of the visualization in a Jupyter notebook.
pyLDAvis.enable_notebook()

#prepares the data for visualization by generating the topic-term distribution, term frequency, and topic frequency distributions using the lda_model, corpus, and id2word.
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=15)


/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [8]:
# save the results in the html
pyLDAvis.save_html(vis, 'hackaton_topic_modeling_fake_news.html')

/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/fuzzysha/software/miniconda3/envs/

/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/fuzzysha/software/miniconda3/envs/hproj/lib/python3.8/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/fuzzysha/software/miniconda3/envs/